## Инструкция:

* Перед выполнением любого скрипта нужно запустить первые два блока **(Общие методы)**: **Работа с `vk_api`** и **Работа со `Spreadsheet API`** (то есть всё с начала и до полупрозрачной линии). Если появляется ошибка `('Connection aborted.', OSError("(54, 'ECONNRESET')",))`, эти два блока нужно перезапустить.

* Чтобы выполнить скрипт, нужно запустить все ячейки, начиная с заголовка скрипта (например, чтобы запустить скрипт **Напоминание о дресс-коде**, нужно выполнить три ячейки, входящие в него). Ячейки выполнять **строго по порядку!**

* Места, куда нужно вписывать текст напоминаний, сообщений и проч., обозначены в коде зелёными комменатриями `# ТЕКСТ ЗДЕСЬ`. Чтобы перейти на новую строку в сообщении, нужно написать символ `\n`.

* Последний раздел, **Технические скрипты**, содержит дополнительные полезные функции. 

## Внимание! Между выполнением скриптов должно пройти не менее 3 мин, иначе скрипт может сломаться во время исполнения.

## Общие методы

### Работа с `vk_api`

In [ ]:
import vk
import time

In [ ]:
# Получает id пользователя из ссылки в формате vk.com/#*#
def getUserId(link):
    try:
        uid = link
        if 'https://' in link:
            uid = link.split('//')[-1]
        if 'vk.com/' in link or 'Vk.com/' in link: #  проверяем эту ссылку
            uid = link.split('/')[-1]  # если да, то получаем его последнюю часть
        if not uid.replace('id', '').isdigit(): # если в нем после отсечения 'id' сами цифры - это и есть id
            uid = vkapi.utils.resolveScreenName(screen_name=uid)['object_id'] # если нет, получаем id с помощью метода API
        else:
            uid = uid.replace('id', '')
        return int(uid)
    except:
        print('Не могу получить id для пользователя ', link)
        return None

In [ ]:
# Аутентификация от имени группы
session = vk.AuthSession(access_token = '#*#')
vkapi = vk.API(session, v = 5.0)

In [ ]:
# Отправка сообщения
def sendMsg(user_id, mesg, attachment = ''):
    try:
        vkapi.messages.send(user_id = user_id, message = mesg, attachment = attachment)
    except:
        print('Не удалось отправить vk.com/id' + str(user_id))

### Работа со `Spreadsheet API`

In [ ]:
# Библиотека для взаимодействия с листом
import gspread 

In [ ]:
# Авторизация
from oauth2client.service_account import ServiceAccountCredentials

scope = ['#*#',
         '#*#']

# Файл с реквизитами "#*#.json" должен лежать в этой же папке
# Не забыть взять из этого файла #*# и расшарить нужный spreadsheet
credentials = ServiceAccountCredentials.from_json_keyfile_name('#*#.json', scope)

gc = gspread.authorize(credentials)

In [ ]:
# Открыть spreadsheet с названием и нужный sheet в нём
sheet = gc.open("#*#").worksheet('#*#')

# Проверка, что открыли то, что нужно
print(sheet.row_values(3))

In [ ]:
# Получить названия листов, с которыми будем работать
def get_sheets():
    name_sheet = gc.open("#*#").worksheet('#*#')
    names = name_sheet.col_values(2)
    names = names[3:]
    return names

---

## Скрипты

### Сообщения

Проверяет, разрешены ли у пользователя сообщения от сообщества, и если они закрыты, направляет второму участнику пары сообщение с напоминанием о необходимости открыть сообщения.

In [ ]:
def messages_allowed_send():
    
    group_id = getUserId('#*#')
    
    pairs_to_send = []
    
    sheet = gc.open("#*#").worksheet('#*#')
    
    fios = sheet.col_values(3)[1:]
    fios = fios.copy()
    pairs = sheet.col_values(1)[1:]
    pairs = pairs.copy()
    links = sheet.col_values(4)[1:]
    links = links.copy()
    
    # Если участнику нельзя отправлять сообщения, добавляем номер его пары в список
    for i in range(len(fios)):
        try:
            if i % 20 == 0:
                time.sleep(2.0)
            pair = pairs[i]
            user_id = getUserId(links[i])
            if vkapi.messages.isMessagesFromGroupAllowed(group_id = group_id, user_id = user_id)['is_allowed'] == 0:
                pairs_to_send.append(pair)
        except:
            print('Возникла проблема со строкой ' + str(i+1) + '. Перехожу на новую строку.')
            continue
            
    # Пройтись по всем участникам, если пара в списке -- попробовать отправить сообщение
    for i in range(len(fios)):
        try:
            if i % 20 == 0:
                time.sleep(2.0)
            if pairs[i] in pairs_to_send:
                if links[i] == 'Нет вк':
                    print('У партнёра нет ВК, пропускаю. Строка ' + str(i+1))
                    continue
                user_id = getUserId(links[i])
                if vkapi.messages.isMessagesFromGroupAllowed(group_id = group_id, user_id = user_id)['is_allowed'] == 1:
                    # ТЕКСТ ЗДЕСЬ
                    sendMsg(user_id, '#*#', attachment=['#*#', '#*#'])
        except:
            print('Возникла проблема со строкой ' + str(i+1) + '. Перехожу на новую строку.')
            continue

In [ ]:
# Запустить напоминание
messages_allowed_send()

### Напоминание об оплате и предоставлении расписки + 8 день

Проверяет, оплатил ли участник участие и/или предоставил ли он расписку, и направляет соответствующее сообщение этому участнику. Также отправляет сообщение на 8 день.

In [ ]:
from datetime import datetime

def remind_about_money():
    
    to_remind_money = []
    to_remind_rasp = []
    to_remind_both = []
    
    # Итерируемся по всем листам 
    names = get_sheets()
    
    for name in names:
        try:
            sheet = gc.open("#*#").worksheet(name)
            fios = sheet.col_values(3)[2:] # Из столбца с фамилией получаем число участников
            fios = fios.copy()
            links = sheet.col_values(4)[2:]
            links = links.copy()
            moneys = sheet.col_values(5)[2:]
            moneys = moneys.copy()
            rasps = sheet.col_values(6)[2:]
            rasps = rasps.copy()
            
            for i in range(len(fios)):
                try:
                    money = moneys[i]
                    rasp = rasps[i]

                    if int(money) == 0 and int(rasp) == 0:
                        to_remind_both.append([fios[i], links[i]])
                    elif int(money) == 0 and int(rasp) == 1:
                        to_remind_money.append([fios[i], links[i]])
                    elif int(money) == 1 and int(rasp) == 0:
                        to_remind_rasp.append([fios[i], links[i]])
                except:
                    print('Не удаётся прочитать строку ' + str(i) + ' на листе ' + str(name) + '. Перехожу на следующую строку.')
                    continue
        except:
            print('Не удаётся прочитать лист ' + str(name) + '. Перехожу на следующий лист.')
            continue
                
    names_sheet = gc.open("#*#").worksheet("#*#")
    names = names_sheet.col_values(2)[2:] + names_sheet.col_values(8)[2:]
    dates = names_sheet.col_values(1)[2:] + names_sheet.col_values(1)[2:]
    
    names = names.copy()
    dates = dates.copy()
    
    moneys = []
    rasps = []
    boths = []
    eights = []
    
    count = 0
    for element in to_remind_money:
        count += 1
        try:
            if count % 20 == 0:
                time.sleep(2.0)
            indx = names.index(element[0])
            date = dates[indx]

            date = date.split(' ')[0]
            datetime_object = datetime.strptime(date, '%d.%m.%Y')
            now = datetime.today()
            delta = now.date() - datetime_object.date()
            days = delta.days

            if days == 1 or days == 4 or days == 6:
                moneys.append([getUserId(str(element[1])), days])
            if days == 8:
                eights.append([getUserId(str(element[1])), days])
        except:
            print('Не удаётся прочитать дату (деньги) для участника ' + str(element[0]) + '. Перехожу к следующему участнику.')
            continue
    
    count = 0
    for element in to_remind_rasp:
        count += 1
        try:
            if count % 20 == 0:
                time.sleep(2.0)
            indx = names.index(element[0])
            date = dates[indx]

            date = date.split(' ')[0]
            datetime_object = datetime.strptime(date, '%d.%m.%Y')
            now = datetime.today()
            delta = now.date() - datetime_object.date()
            days = delta.days

            if days == 1 or days == 4 or days == 6:
                rasps.append([getUserId(str(element[1])), days])
            if days == 8:
                eights.append([getUserId(str(element[1])), days])
        except:
            print('Не удаётся прочитать дату (расписка) для участника ' + str(element[0]) + '. Перехожу к следующему участнику.')
            continue
            
    count = 0
    for element in to_remind_both:
        count += 1
        try:
            if count % 20 == 0:
                time.sleep(2.0)
            indx = names.index(element[0])
            date = dates[indx]

            date = date.split(' ')[0]
            datetime_object = datetime.strptime(date, '%d.%m.%Y')
            now = datetime.today()
            delta = now.date() - datetime_object.date()
            days = delta.days

            if days == 1 or days == 4 or days == 6:
                boths.append([getUserId(str(element[1])), days])
            if days == 8:
                eights.append([getUserId(str(element[1])), days])
        except:
            print('Не удаётся прочитать дату (деньги и расписка) для участника ' + str(element[0]) + '. Перехожу к следующему участнику.')
            continue
    
    count = 0
    for element in moneys:
        count += 1
        if count % 20 == 0:
            time.sleep(2.0)
        # ТЕКСТ ЗДЕСЬ
        message = '#*# ' + str(int(7 - element[1])) + ' #*#'
        sendMsg(element[0], message)
    
    count = 0
    for element in rasps:
        count += 1
        if count % 20 == 0:
            time.sleep(2.0)
        # ТЕКСТ ЗДЕСЬ
        message = '#*# ' + str(int(7 - element[1])) + ' #*#'
        sendMsg(element[0], message, '#*#')
    
    count = 0
    for element in boths:
        count += 1
        if count % 20 == 0:
            time.sleep(2.0)
        # ТЕКСТ ЗДЕСЬ
        message = '#*# ' + str(int(7 - element[1])) + ' #*#'
        sendMsg(element[0], message, '#*#')
    
    count = 0
    for element in eights:
        count += 1
        if count % 20 == 0:
            time.sleep(2.0)
        # ТЕКСТ ЗДЕСЬ
        message = '#*#'
        sendMsg(element[0], message)

In [ ]:
# Запустить напоминание об оплате и расписке
remind_about_money()

### Напоминание о дресс-коде

Просматривает каждого участника и отправляет ему уведомление о тех элементах дресс-кода, фото которых он ещё не прислал. Отправка происходит в две партии: сначала людям на первых 7 листах, после – ожидание 100 сек., затем – отправка людям на вторых 7 листах.

In [ ]:
# Список с позициями дресс-кода
dresses_glob = ["#*#\n", "#*#\n", "#*#\n", "#*#\n", "#*#\n", "#*#\n"]

In [ ]:
import numpy as np 

def remind_about_dresscode(names):
    # Итерируемся по всем листам
    names = names
    
    for name in names:
        sheet = gc.open("#*#").worksheet(name)
        fios = sheet.col_values(3)[2:] # Из столбца с фамилией получаем число участников
        fios = fios.copy()
        links = sheet.col_values(4)[2:]
        links = links.copy()
        is_allowed = sheet.col_values(12)[2:]
        is_allowed = is_allowed.copy()
        #*# = sheet.col_values(13)[2:]
        #*# = #*#.copy()
        #*# = sheet.col_values(14)[2:]
        #*# = #*#.copy()
        #*# = sheet.col_values(15)[2:]
        #*# = #*#.copy()
        #*# = sheet.col_values(16)[2:]
        #*# = #*#.copy()
        #*# = sheet.col_values(17)[2:]
        #*# = #*#.copy()
        #*# = sheet.col_values(18)[2:]
        #*# = #*#.copy()
        #*# = [#*#, #*#, #*#, #*#, #*#, #*#]

        for i in range(len(fios)):
            try:
                if i % 20 == 0:
                    time.sleep(2.0)
                dress_code = [] # Хранит индексы позиций с 0, т.е. о которых нужно напомнить
                if int(is_allowed[i]) != 0: # Проверяем, можно ли отправлять сообщения
                    for p in range(len(code)): # Индексы позиций
                        if int(code[p][i]) == 0:
                            dress_code.append(p)
                    if np.sum(np.array(dress_code)) != 0: # Если все нули, не напоминаем
                        # ТЕКСТ ЗДЕСЬ
                        msg = '#*#\n'
                        for element in dress_code:
                            msg = msg + dresses_glob[element]
                        uid = getUserId(links[i])
                        sendMsg(uid, msg)
            except:
                print('Не могу прочитать строку ' + str(i) + ' на листе ' + str(name) + '. Перехожу на следующую строку.')
                continue

In [ ]:
# Запустить напоминание для первых 7 листов
remind_about_dresscode(get_sheets()[:7])
print('Первые 7 листов пройдены!')
# Ждать 100 сек.
time.sleep(100.0)
# Запустить напоминание для вторых 7 листов
remind_about_dresscode(get_sheets()[7:])

### Рассылка

Делает рассылку сообщения по всем участникам. Чтобы перейти на новую строку в сообщении, нужно написать символ `\n`.

In [ ]:
def mailing(message):
    sheet = gc.open("#*#").worksheet('#*#')
    
    fios = sheet.col_values(3)[1:]
    fios = fios.copy()
    links = sheet.col_values(4)[1:]
    links = links.copy()
    
    for i in range(len(fios)):
        if i % 20 == 0:
            time.sleep(2.0)
        sendMsg(getUserId(links[i]), message)

In [ ]:
# Запустить рассылку
message = '#*#'
mailing(message) # ТЕКСТ ЗДЕСЬ

### Напоминание об опросе МК

Отправляет участникам, которые оплатили участие, но не прошли опрос, напоминание об опросе.

In [ ]:
def remind_mk():
    sheet = gc.open("#*#").worksheet('#*#')
    
    fios = sheet.col_values(3)[1:]
    fios = fios.copy()
    links = sheet.col_values(4)[1:]
    links = links.copy()
    pays = sheet.col_values(6)[1:]
    pays = pays.copy()
    mks = sheet.col_values(7)[1:]
    mks = mks.copy()
    
    # ТЕКСТ ЗДЕСЬ
    message = '#*#'
    
    for i in range(len(fios)):
        if i % 20 == 0:
            time.sleep(2.0)
        if int(pays[i]) == 1 and int(mks[i]) == 0:
            sendMsg(getUserId(links[i]), message)

In [ ]:
# Запустить напоминание об опросе МК
remind_mk()

### Напоминание неоплатившим

Отправляет напоминание участникам, которые не оплатили участие.

In [ ]:
def remind_not_paid():
    sheet = gc.open("#*#").worksheet('#*#')
    
    fios = sheet.col_values(3)[1:]
    fios = fios.copy()
    links = sheet.col_values(4)[1:]
    links = links.copy()
    pays = sheet.col_values(6)[1:]
    pays = pays.copy()
    
    # ТЕКСТ ЗДЕСЬ
    message = '#*#'
    
    for i in range(len(fios)):
        if i % 20 == 0:
            time.sleep(2.0)
        if int(pays[i]) == 0:
            sendMsg(getUserId(links[i]), message)

In [ ]:
# Запустить напоминание неоплатившим
remind_not_paid()

### Рассылка дат по МК (без проверок, отправляли ли уже)

Проходит по всем листам кураторов и отправляет информацию о датах мастер-классов участникам.

In [ ]:
def send_dates(names_list):
    # Итерируемся по всем листам
    names = names_list
    names = names.copy()
    
    for name in names:
        try:
            sheet = gc.open("#*#").worksheet(name)
            fios = sheet.col_values(3)[2:] # Из столбца с фамилией получаем число участников
            fios = fios.copy()
            links = sheet.col_values(4)[2:]
            links = links.copy()
            mk_1 = sheet.col_values(8)[2:]
            mk_1 = mk_1.copy()
            mk_2 = sheet.col_values(9)[2:]
            mk_2 = mk_2.copy()
            mk_3 = sheet.col_values(10)[2:]
            mk_3 = mk_3.copy()
            mk_4 = sheet.col_values(11)[2:]
            mk_4 = mk_4.copy()
            gen_rep = sheet.col_values(19)[2:]
            gen_rep = gen_rep.copy()

            for i in range(len(fios)):
                try:
                    if i % 6 == 0:
                        time.sleep(2.0)

                    if mk_1[i] != '':
                        message = '#*#\n#*# - ' +
                        str(mk_1[i]) + ';\n#*# - ' +
                        str(mk_2[i]) + ';\n#*# - ' +
                        str(mk_3[i]) + ';\n#*# - ' +
                        str(mk_4[i]) + ';\n#*# - ' +
                        str(gen_rep[i]) + '.\n#*#'
                        sendMsg(getUserId(links[i]), message)
                    else:
                        continue
                except:
                    print('Проблема со строкой ' + str(i) + ' на листе ' + str(name) + '. Перехожу на следующую строку.')
                    continue
        except:
            print('Не могу прочитать лист: ' + str(name) + '. Перехожу на следующий лист.')
            continue

In [ ]:
# Запустить напоминание для первых 7 листов
send_dates(get_sheets()[:7])
print('Первые 7 листов пройдены!')
# Ждать 100 сек.
time.sleep(100.0)
# Запустить напоминание для вторых 7 листов
send_dates(get_sheets()[7:])

---

## Технические скрипты

### Сгенерировать словарь {Участник: [Организатор, Имя Фамилия участника]}

In [ ]:
# В словаре хранятся только числовые id, без приставки!
def generate_orgs_vocab():
    
    name_sheet = gc.open("#*#").worksheet('#*#')
    names = name_sheet.col_values(2)[3:]
    names = names.copy()
    adresses = name_sheet.col_values(3)[3:]
    adresses = adresses.copy()
    
    sheet = gc.open("#*#").worksheet('#*#')
    
    fios = sheet.col_values(3)[1:]
    links = sheet.col_values(4)[1:]
    orgs = sheet.col_values(5)[1:]
    
    fios = fios.copy()
    links = links.copy()
    orgs = orgs.copy()
    
    vocab = dict()
    for i in range(len(fios)):
        try:
            if i % 20 == 0:
                time.sleep(2.0)
            partic_id = str(getUserId(links[i]))
            partic_name = str(fios[i])

            org_name = orgs[i]
            org_adress = adresses[names.index(org_name)]

            org_id = str(getUserId(org_adress))

            vocab[partic_id] = [org_id, partic_name]
        except:
            print('Не могу прочитать строку ' + str(i) + '. Перехожу на следующую строку.')
            continue
    print(vocab)

In [ ]:
# Запустить генератор
generate_orgs_vocab()

### Проверить, разрешены ли у пользователя сообщения от сообщества

In [ ]:
def check_if_messaging_allowed(link):
    group_id = getUserId('#*#')
    user_id = getUserId(link)
    if vkapi.messages.isMessagesFromGroupAllowed(group_id = group_id, user_id = user_id)['is_allowed'] == 1:
        print('Разрешены')
    else:
        print('Не разрешены')

In [ ]:
# Проверить: в аргумент ввести ссылку в формате vk.com/#*#
check_if_messaging_allowed('vk.com/#*#')